In [2]:
import pandas as pd
import lightgbm as lg
import sklearn as sk
from collections import Counter
import re

In [42]:
tables = ["customers", "merchants", "receipts", "terminals", "transactions"]

dfs = {}

for name in tables:
    dfs[name] = pd.read_csv(f"data/{name}.csv")
    dfs[name].dropna(inplace=True)
    dfs[name].info()
    print("=" * 100)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   customer_id       500 non-null    object
 1   customer_segment  500 non-null    object
dtypes: object(2)
memory usage: 7.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   merchant_id    150 non-null    object
 1   merchant_name  150 non-null    object
 2   merchant_city  150 non-null    object
dtypes: object(3)
memory usage: 3.6+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 3710 entries, 0 to 4475
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   transaction_id  3710 non-null   object 
 1   item_position   3710 non-null   int64  
 2   item_name       3710 non-nu

In [46]:
to_format = [
    (dfs["terminals"], "terminal_description"),
    (dfs["receipts"], "item_name")
]

def clean_text(s):
    s = str(s)

    s = re.sub(r"[^\w\s]", "", s)
    s = re.sub(r" [a-zA-Z] ", "", s)
    s = s.strip()
    return s.lower()

for df, col in to_format:
    df[col] = df[col].apply(clean_text)

In [47]:
dfs["terminals"][["terminal_description"]]

,terminal_description
0,common common common thing
1,binging hoes close eto basic
2,stuff service stuff services staples
3,basic basic basic basics stapleas
4,bringng common clozer common
...,...
293,stuff stuff at stuff common
294,common common commoncommon
295,service
296,regular regular vivilylifee ere


In [53]:
def sim(s1, s2):
    if len(s1) > len(s2):
        s1, s2 = s2, s1

    distances = range(len(s1) + 1)
    for i2, c2 in enumerate(s2):
        distances_ = [i2 + 1]
        for i1, c1 in enumerate(s1):
            if c1 == c2:
                distances_.append(distances[i1])
            else:
                distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
        distances = distances_
    return distances[-1]

dict = []



['0', 'common', 'common', 'common', 'thing', '1', 'binging', 'hoes', 'close', 'eto', 'basic', '2', 'stuff', 'service', 'stuff', 'services', 'staples', '3', 'basic', 'basic', 'basic', 'basics', 'stapleas', '4', 'bringng', 'common', 'clozer', 'common', '...', '293', 'stuff', 'stuff', 'at', 'stuff', 'common', '294', 'common', 'common', 'commoncommon', '295', 'service', '296', 'regular', 'regular', 'vivilylifee', 'ere', '297', 'a', 'thing', 'general', 'general', 'Name:', 'terminal_description,', 'Length:', '296,', 'dtype:', 'object']
['0', 'braenk', '1', 'etuers', '4', 'unfoldiin', 'gamber', '6', 'unit', 'part', 'stuff', '7', 'element', 'part', '...', '4471', 'thing', 'unefoldign', '4472', 'aurleiuc', 'thing', '4473', 'petirc', 'object', '4474', 'piece', 'stuff', '4475', 'item', 'Name:', 'item_name,', 'Length:', '3710,', 'dtype:', 'object']


In [54]:
from collections import defaultdict

columns = [
    (dfs["terminals"], "terminal_description"),
    (dfs["receipts"], "item_name"),
]

all_words = set()

for df, col in columns:
    df["_words"] = df[col].str.split()
    for row in df["_words"]:
        for w in row:
            if len(w) > 1:
                all_words.add(w)

all_words = list(all_words)

In [55]:
all_words

['basic',
 'sarenaude',
 'gatherng',
 'carulean',
 'fargment',
 'threaedd',
 'ay',
 'stlli',
 'reverioe',
 'torning',
 'glded',
 'etherboun',
 'oframents',
 'greslove',
 'ujourney',
 'veiins',
 'usrroundings',
 'embraca',
 'oharizonscape',
 'bireach',
 'asheore',
 'emervalut',
 'eseawys',
 'tiudweay',
 'lsonnet',
 'reveie',
 'taepstury',
 'whespering',
 'ohllows',
 'pound',
 'alluore',
 'fracteured',
 'orifdborne',
 'shadowinag',
 'aollure',
 'uminous',
 'tapesotry',
 'resolvee',
 'units',
 'niklin',
 'aowakened',
 'tharead',
 'peal',
 'eimbervault',
 'monthead',
 'wituh',
 'hearmonized',
 'teps',
 'celestieal',
 'eombervaulit',
 'texutures',
 'sweay',
 'race',
 'dethesr',
 'at',
 'desinged',
 'moion',
 'rtace',
 'ibrecah',
 'holows',
 'csacade',
 'farost',
 'veeins',
 'swey',
 'sanonet',
 'fargments',
 'obsidina',
 'cescent',
 'tideya',
 'sourge',
 'aisffron',
 'mebered',
 'froegment',
 'ahsoer',
 'whiespoers',
 'aureilc',
 'boeundless',
 'aweakneed',
 'threa',
 'fragmeents',
 'revebr

In [56]:
index = defaultdict(list)

for w in all_words:
    key = (len(w), w[0])
    index[key].append(w)

In [57]:
parent = {}

def find(x):
    parent.setdefault(x, x)
    if parent[x] != x:
        parent[x] = find(parent[x])
    return parent[x]

def union(a, b):
    pa, pb = find(a), find(b)
    if pa != pb:
        parent[pb] = pa

In [60]:
for w in all_words:
    for dl in (-2, -1, 0, 1, 2):
        key = (len(w) + dl, w[0])
        if key not in index:
            continue

        for candidate in index[key]:
            if candidate == w:
                continue

            if abs(len(w) - len(candidate)) > 2:
                continue

            if sim(w, candidate) < 2:
                union(w, candidate)

In [62]:
parent

{'basic': 'basic',
 'basics': 'basic',
 'sarenaude': 'sihift',
 'srenade': 'stillensis',
 'serenaide': 'stillensis',
 'carulean': 'ceroelan',
 'cerlean': 'ceroelan',
 'ceruleen': 'ceroelan',
 'cerulaan': 'ceroelan',
 'caruleian': 'ceroelan',
 'caruleuan': 'ceroelan',
 'fargment': 'fragant',
 'fagment': 'fragant',
 'frament': 'fragant',
 'fargemnt': 'fragant',
 'frgament': 'fragant',
 'fregment': 'fragant',
 'farments': 'fragant',
 'fagments': 'fragant',
 'fragment': 'fragant',
 'fargments': 'fragant',
 'fargument': 'fragant',
 'fargmented': 'fragant',
 'threaedd': 'trever',
 'threed': 'trever',
 'threead': 'trever',
 'threaad': 'trever',
 'threadd': 'trever',
 'threatd': 'trever',
 'threded': 'trever',
 'threaod': 'trever',
 'threeed': 'trever',
 'thraeded': 'trever',
 'threadad': 'trever',
 'threedde': 'trever',
 'threeded': 'trever',
 'threaeded': 'trever',
 'ay': 'asoile',
 'at': 'asoile',
 'ad': 'asoile',
 'af': 'asoile',
 'ato': 'asoile',
 'aof': 'asoile',
 'auf': 'asoile',
 'stll

In [63]:
word_to_id = {}
next_id = 1

for w in all_words:
    root = find(w)
    if root not in word_to_id:
        word_to_id[root] = next_id
        next_id += 1

In [64]:
def words_to_ids(words):
    ids = set()
    for w in words:
        if len(w) <= 1:
            continue
        root = find(w)
        ids.add(word_to_id[root])
    return list(ids)

for df, col in columns:
    df[col + "_ids"] = df["_words"].apply(words_to_ids)
    df.drop(columns="_words", inplace=True)

In [69]:
dfs["receipts"].head(100)

,transaction_id,item_position,item_name,item_price,item_name_ids
0,TX00001116,1,braenk,58.20,[19]
1,TX00001116,2,etuers,28.99,[17]
4,TX00001368,2,unfoldiin gamber,25.44,"[232, 84]"
6,TX00000422,2,unit part stuff,85.41,"[418, 34, 31]"
7,TX00000422,3,element part,39.34,"[152, 34]"
...,...,...,...,...,...
111,TX00000076,4,piece ecilpse,2.41,"[17, 34]"
112,TX00001391,1,vains,89.16,[15]
113,TX00001391,2,item,52.04,[61]
114,TX00001391,3,element,88.74,[152]


In [77]:
x_train = dfs["terminals"][["terminal_description_ids"]].copy()
x_train.head()
x_train[["item_name_ids"]] = dfs["receipts"][["item_name_ids"]]
x_train.info(100)
#y_train =

#x_val =
#y_val =

<class 'pandas.core.frame.DataFrame'>
Index: 296 entries, 0 to 297
Data columns (total 2 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   terminal_description_ids  296 non-null    object
 1   item_name_ids             239 non-null    object
dtypes: object(2)
memory usage: 6.9+ KB
